# Parse dataset into dict

In [8]:
def parse_txt(filename, oriented=True):
    '''
    Via sets, but it is not JSON serializable
    '''
    data = {}
    with open(filename) as file:
        
        line = file.readline()
        while line:
            
            # skip comments
            if line[0] == '#':
                line = file.readline()
                continue
            
            parent, child = line.split()
            parent = int(parent)
            child = int(child)
            
            # rows in data file can be duplicated
            if parent in data:
                data[parent]['linked'].add(child)
                data[parent]['degree'] += 1
            else:
                data[parent] = {
                    'linked': {child},
                    'degree': 1,
                    'marked': False,
                    'centrality': 0
                }
            
            # add reversed link on parent vertice for not oriented graph
            if not oriented:
                if child in data:
                    data[child]['linked'].add(parent)
                    data[child]['degree'] += 1
                else:
                    data[child] = {
                        'linked': {parent},
                        'degree': 1,
                        'marked': False,
                        'centrality': 0
                    }

            line = file.readline()
            
    return data


def parse(filename, oriented=True):
    if filename.split('.')[-1] == 'txt':
        return parse_txt(filename, oriented)

In [9]:
def parse_txt(filename, oriented=True):
    '''
    Via lists
    '''
    data = {}
    with open(filename) as file:
        
        line = file.readline()
        while line:
            
            # skip comments
            if line[0] == '#':
                line = file.readline()
                continue
            
            parent, child = line.split()
            parent = int(parent)
            child = int(child)
            
            # rows in data file can be duplicated
            if parent in data:
                if child not in data[parent]['linked']:
                    data[parent]['linked'].append(child)
                    data[parent]['degree'] += 1
            else:
                data[parent] = { 
                    'linked': [child],
                    'distances': {},
                    'degree': 1,
                    'marked': False,
                    'centrality': 0
                }
                
            if not oriented:
                if child in data:
                    if parent not in data[child]['linked']:
                        data[child]['linked'].append(parent)
                        data[child]['degree'] += 1

                else:    
                    data[child] = {
                        'linked': [parent],
                        'distances': {},
                        'degree': 1,
                        'marked': False,
                        'centrality': 0
                    }

            line = file.readline()
            
    return data
def parse(filename, oriented=True):
    if filename.split('.')[-1] == 'txt':
        return parse_txt(filename, oriented)

In [10]:
from collections import deque 

def count_distance(vertice, data):
    '''
    counts distances form given vertice to all other in connectivity component that vertice belongs to
    based on BFS
    vertice: index of source vertice
    data: dict with information about graph
    '''
    current_distance = 0
    centrality = 0
    vertices_number = 1
    d0 = deque()
    d1 = deque()
    
    d0.append(vertice)
    data[vertice]['marked'] = True
    
    while True:
        
        if not d0 and not d1:
            break
        
        if current_distance % 2 == 0:
            
            v = d0.pop()
            
            data[v]['distances'][vertice] = current_distance
            vertices_number += 1
            centrality += current_distance
                
            for i in data[v]['linked']:
                if not data[i]['marked']:
                    d1.append(i)
                    data[i]['marked'] = True
            
            if not d0:
                current_distance += 1
                
        else:
            
            v = d1.pop()

            data[v]['distances'][vertice] = current_distance
            vertices_number += 1
            centrality += current_distance
                
            for i in data[v]['linked']:
                if not data[i]['marked']:
                    d0.append(i)
                    data[i]['marked'] = True
            
            if not d1:
                current_distance += 1

        
    for key, value in data.items():
        value['marked'] = False
    
    data[vertice]['centrality'] = centrality / vertices_number

    
def count_distances(data):
    for key, value in data.items():
        count_distance(key, data)
        

def bfs(source, stock, data):
    '''
    Count distance from source to stock.
    '''
    current_distance = 0
    d0 = deque()
    d1 = deque()
    
    d0.append(source)
    data[source]['marked'] = True
    
    while True:
        
        if not d0 and not d1:
            current_distance = -1
            break
        
        if current_distance % 2 == 0:
            
            v = d0.pop()
            
            if v == stock:
                break
                
            for i in data[v]['linked']:                
                if not data[i]['marked']:
                    d1.append(i)
                    data[i]['marked'] = True
            
            if not d0:
                current_distance += 1
                
        else:
            
            v = d1.pop()
            
            if v == stock:
                break
                
            for i in data[v]['linked']:
                if not data[i]['marked']:
                    d0.append(i)
                    data[i]['marked'] = True
            
            if not d1:
                current_distance += 1 
        
    for key, value in data.items():
        value['marked'] = False
        
    return current_distance

In [ ]:
d = {
    1: {
        'linked': [2,3,4,10],
        'distances':{},
        'marked': False
    },
    2: {
        'linked': [1,5,6,7],
        'distances':{},
        'marked': False
    },
    3: {
        'linked': [1,8],
        'distances':{},
        'marked': False
    },
    4: {
        'linked': [1,9,10],
        'distances':{},
        'marked': False
    },
    5: {
        'linked': [2],
        'distances':{},
        'marked': False
    },
    6: {
        'linked': [2],
        'distances':{},
        'marked': False
    },
    7: {
        'linked': [2],
        'distances':{},
        'marked': False
    },
    8: {
        'linked': [3],
        'distances':{},
        'marked': False
    },
    9: {
        'linked': [4],
        'distances':{},
        'marked': False
    },
    10: {
        'linked': [4,1],
        'distances':{},
        'marked': False
    }
}

print(bfs(1, 3, d))

In [11]:
data = parse('test.txt', oriented=False)

In [20]:
print(data)

{84424: {'linked': [276, 1662, 5089, 6058, 6229, 10639, 16442, 19325, 19834, 20113, 21937, 25452, 26902, 29829, 30222, 32432, 33040, 39238, 39521, 41418, 45009, 45098, 45242, 47005, 47968, 47999, 49934, 50220, 50897, 51730, 53681, 57537, 58458, 59326, 61571, 63552, 64124, 64568, 66200, 69839, 72391, 73543, 76259, 77098, 77915, 78627, 83560, 85420, 88768, 89131, 89308, 89994, 90506, 91060, 92387, 93296, 94138, 94329, 95070, 95531, 96570, 97101, 98506, 99104, 104802, 106611, 107829, 109016, 112605, 117751, 122908, 124023, 125190, 130825, 132445], 'distances': {82704: 3, 77208: 3, 99023: 3, 25095: 3, 124913: 3, 100099: 3, 33040: 1, 8381: 3, 106274: 2, 90506: 1, 39238: 1, 104241: 3, 115420: 3, 30400: 4, 64124: 1, 4239: 3, 103169: 3, 43227: 3, 276: 1, 12203: 3, 24075: 2, 21729: 3, 93271: 3, 60310: 2, 67716: 3, 5099: 3, 69459: 4, 76706: 3, 33777: 2, 107829: 1, 117751: 1, 811: 3, 113138: 2, 99870: 2, 15829: 2, 71722: 3, 27124: 3, 3365: 2, 26526: 2, 45242: 1, 95261: 3, 42994: 2, 75607: 2, 5089

# Select landmarks random

In [20]:
import random

data_items = data.items()
vertices = [i[0] for i in data_items]

graph_size = len(data_items)
percent_of_landmarks = 10
number_of_landmarks = int(graph_size * (percent_of_landmarks / 100))

random_indexes = random.sample(range(0,graph_size), number_of_landmarks)
landmarks = [vertices[i] for i in random_indexes]

print(graph_size)
print(number_of_landmarks)
print(landmarks)

186
18
[125345, 63769, 50991, 45098, 95531, 62496, 37861, 69582, 276, 45009, 32432, 112243, 95860, 72233, 68320, 55528, 94235, 28787]


# Select landmarks using their degree

In [3]:
data_sorted = sorted(data.items(), key=lambda x: x[1]['degree'], reverse=True)
graph_size = len(data_sorted)

percent_of_landmarks = 10
number_of_landmarks = int(graph_size * (percent_of_landmarks / 100))

landmarks = [i[0] for i in data_sorted[:number_of_landmarks]]

print(graph_size)
print(number_of_landmarks)
print(landmarks)
# print(data_sorted)

186
18
[94138, 84424, 89308, 63225, 73543, 21937, 61571, 64124, 89994, 104802, 109016, 276, 1662, 5089, 6058, 6229, 10639, 16442]


# Select landmarks using their closeness centrality

In [28]:
import random

data_items = data.items()
vertices = [i[0] for i in data_items]

graph_size = len(data_items)

percent_of_seeds = 20
number_of_seeds = int(graph_size * (percent_of_seeds / 100))
print('Number of seeds: ' + str(number_of_seeds))

percent_of_landmarks = 10
assert percent_of_landmarks <= percent_of_seeds
number_of_landmarks = int(graph_size * (percent_of_landmarks / 100))
print('Number of landmarks: ' + str(number_of_landmarks))

# select random seeds from input graph to calculate their closeness centrality
random_indexes = random.sample(range(0,graph_size), number_of_landmarks)
seeds = [vertices[i] for i in random_indexes]

for seed in seeds:
    count_distance(seed, data)

landmarks = [
    i[0] for i in sorted(
        [(j[0], j[1]['centrality']) for j in data.items() if j[0] in seeds],
        key= lambda x: x[1],
        reverse=True
    )[:number_of_landmarks]
]
print('Total number of landmarks: ' + str(len(landmarks)))
print(landmarks)
# for l in landmarks:
#     print(str(l) + ":")
#     print(data[l])

Number of seeds: 87
Number of landmarks: 43
Total number of landmarks: 43
[99504, 15406, 29421, 31334, 51604, 67988, 70637, 102617, 57772, 560, 57618, 116343, 6256, 31235, 5335, 56117, 58532, 61461, 61686, 61929, 65794, 74612, 101953, 41418, 45242, 51730, 98506, 107829, 3335, 18007, 44183, 56361, 71801, 75690, 94560, 100717, 103169, 112848, 119042, 125678, 73543, 26713, 104802]
